In [307]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import clear_output

sns.set_style('darkgrid')

In [333]:
num_trials = 100000

num_dice = 1
dice_size = 12

rules = {"drop_highest": False ,
         "drop_lowest" : False }

def summarize(arr, num_dice=4, dice_size=6, num_trials=100000, rules=rules):
    rules2 = [k.replace('_'," ") for k in rules if rules[k]]
    rules2 = ", ".join(rules2)
    if len(rules2)>0:
        rules2 = "("+rules2+")"
    print("{0}d{1} for {2} trials {3}".format(num_dice,dice_size,num_trials,rules2))
    print("Min  :",arr.min())
    print("Mean :",arr.mean())
    print("+/-  :",arr.std())
    print("Max  :",arr.max())
    
    num_bins = np.arange(0, dice_size*num_dice)
    
    plt.figure(figsize=(12,6))
    ax = sns.countplot(arr)
    plt.xlabel('Total')
    plt.ylabel('Frequency')
    plt.title("{0}d{1} {2}".format(num_dice,dice_size,rules2))
    
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height + 30,
                '{:1.1f}%'.format(100*height/num_trials),
                ha="center") 
    plt.show()

def calculate(num_dice, dice_size, num_trials, rules):
    combs = np.random.randint(1, high=dice_size+1, size=(num_trials,num_dice))

    if rules["drop_lowest"]:
        combs[np.arange(0,num_trials),combs.argmin(axis=1)] = 0
    if rules["drop_highest"]:
        combs[np.arange(0,num_trials),combs.argmax(axis=1)] = 0

    combs = combs.sum(axis=1)

    summarize(combs, num_dice, dice_size, num_trials, rules)
    
# calculate(num_dice, dice_size, num_trials, rules)

In [357]:
%matplotlib inline

w_dl_label   = widgets.Label('Drop Lowest')
w_drop_l = widgets.Checkbox(False)
w_dh_label   = widgets.Label('Drop Highest')
w_drop_h = widgets.Checkbox(False)

w_num_dice  = widgets.BoundedIntText(value=1,min=1,max=100,step=1,
                                    layout=widgets.Layout(width='50px', height='40px'))
w_d_label   = widgets.Label('d')
w_dice_size = widgets.BoundedIntText(value=6,min=1,max=100,step=1,
                                    layout=widgets.Layout(width='50px', height='40px'))

w_submit = widgets.Button(description='Go', button_style='success')
out      = widgets.Output()

def submit_func(b):
    out.clear_output()
    with out:
        num_dice = w_num_dice.value
        dice_size = w_dice_size.value
        rules = {"drop_highest": w_drop_h.value ,"drop_lowest" : w_drop_l.value}
        
        calculate(num_dice, dice_size, num_trials, rules)
        
        
w_submit.on_click(submit_func)

front_end = widgets.VBox([widgets.HBox([w_dl_label, w_drop_l]),
                          widgets.HBox([w_dh_label, w_drop_h]),
                          widgets.HBox([w_num_dice, w_d_label, w_dice_size]),
                          w_submit,
                          out])

clear_output()
display(front_end)